In [4]:
test_ids = []
test_contents = []
test_keywords = []
with open("./testData.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        test_ids.append(line.split(" ")[0])
        test_contents.append(line.split(" ")[1])
        test_keywords.append(line.split(" ")[2])

In [6]:
print(len(test_ids))
print(len(test_contents))
print(len(test_keywords))

1000
1000
1000


In [1]:
'''
    zsygzh	Qa123456
'''

from aip import AipNlp

APP_ID = '*********'
API_KEY = '******************'
SECRET_KEY = '**************************'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [2]:
def cutSentences(text):
    
    import time
    
    cutwords = []

    if text is None:
        return None

    result = client.lexerCustom(text)

    items = result.get('items', [])
    items_size = len(items)

    tries_limit = 3
    tries_counter = 0
    
    while items_size == 0: # 分词结果为空
        if len(text) != 0: # 但句子长度不为空
            time.sleep(1)
            result = client.lexerCustom(text)

            items = result.get('items', [])
            items_size = len(items)

        tries_counter += 1

        if tries_counter >= tries_limit: # 分词尝试大于等于4次之后仍失败
            return None
        
    restore_idx = 0
    
    while restore_idx < items_size: # 对每个分词结果进行整理 
        cutwords.append(items[restore_idx]['item'])
        restore_idx += 1 # 继续下一个item
    
    return cutwords

In [10]:
import re
with open("./cut_testData.txt", "w", encoding="utf-8") as result:
    for num, sent in enumerate(test_contents): # 每篇文档
        sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk').strip()
        try:
            result.write(" ".join(cutSentences(sent))+"\n")
        except Exception as e:
            print(num)
            print(e)

In [14]:
with open("./cut_testData.txt", "r", encoding="utf-8") as result:
    cutTestData = result.readlines()

In [94]:
stopwords_list = []
with open("./stop_words.txt", "r", encoding="utf-8") as stopwords:
    for line in stopwrds:
        line = line.strip()
        stopwords_list.append(line)

FileNotFoundError: [Errno 2] No such file or directory: './stop_words.txt'

In [58]:
add_title_weights_cutTestData = []
for line in cutTestData:
    line = line.strip()
    line += re.split("。", line)[0]*5
    add_title_weights_cutTestData.append(line)

In [96]:
add_title_weights_cutTestData[0]

'《 三生三世 》 里 凤九东华 最 有 “ 夫妻相 ” ， 这 一模一样 的 动作 是 证明 。 电视剧 《 三生三世十里桃花 》 结束了 ， 接下来 它 的 姊妹篇 《 三生三世枕上书 》 又要 开拍 了 。 上 一部 讲述 的 是 杨幂 出演 的 白浅 与 赵又廷 出演 的 夜华 。 而 这 一部 却 讲述 了 迪丽热巴 出演 的 白凤九 与 高伟光 出演 的 东华帝君 。 前 一部 里面 还是 两大 配角 ， 这 一部 剧 却 成 了 主角 。 不过 她们 的 故事 在 《 三生三世十里桃花 》 里面 已经 讲述 了 一大半 了 ， 再 到 《 三生三世枕上书 》 里面 来 再 经历 一次 三生三世 ， 也 着实 让 人 为 剧情 捏 了 一把 汗 。 不得不 说 的 是 ， 就算 是 原著 作品 ， 《 三生三世十里桃花 》 的 剧情 构造 都是 要 比 《 三生三世枕上书 》 来 的 精彩 的 。 而且 很多 情节 ， 都 有 “ 借鉴 ” 他人 作品 的 嫌疑 。 原 班 人马 出发 ， 固然 是 对 喜爱 这部 作品 的 观众 的 尊重 ， 但是 对于 演员 本身 却 没有 多大 的 好处 。 就像 刚刚 公布 男女 主 阵容 的 时候 ， 迪丽热巴 的 粉丝 就 产生 了 极大 的 抵触 ， 认为 她 家 公司 只 知道 圈钱 ， 却 不知道 为 迪丽热巴多 接 一些 好 的 作品 ， 毕竟 迪丽热巴 同类型 的 古装 剧 实在 是 太多 了 ， 需要 一部 真正 的 大女 主 ， 来 巩固 一线 位置 。 做 人气 小花 容易 ， 做 长久 的 女 演员 却是 很难 的 。 不过 ， 再 看 《 三生三世十里桃花 》 这 一部 无 论 口碑 还是 制作 都是 上良 的 剧集 ， 不得不 说 ， 编剧 在 很多 小 细节 上面 的 把控 是 很 到位 的 。 今天 就 来说 一说 ， 凤九 与 东华 之间 那些 暗 搓 搓 的 秀恩爱 。 《 三生三世 》 里 凤九东华 最 有 “ 夫妻相 ” ， 这个 动作 简直 粘贴 复制 。 首先 这 一组 托 晒 沉思 的 动作 ， 简直 是 粘贴 复制 的 存在 啊 ！ 还有 凤九 受伤 之后 的 拥抱 ， 东华帝君 的 臂膀 太 宽大 了 ， 感觉 抱着 一个 小孩 一样 ， 很 宠溺 ， 很 甜 有没

### tfidf

In [43]:
import math
def train_idf(doc_list):
    idf_dic = {}
    tt_count = len(doc_list) # 获取整个文档集的长度
    
    # 在整个文档集中，每个单词出现的次数
    for doc in doc_list:
        for word in set(doc.split(" ")):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
    
    # 按公式计算idf值，分母加1平滑处理
    for k,v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (v+1.0))
    
    # 没在字典中的字，默认仅在一个文档中出现
    default_idf = math.log(tt_count / 1.0)
    
    return idf_dic, default_idf

In [44]:
# 统计tf值
def train_tf(word_list):
    tf_dic = {}
    for word in word_list.split(" "):
        tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
    tt_count = len(word_list.split(" "))
    for k,v in tf_dic.items():
        tf_dic[k] = float(v) / tt_count
    return tf_dic

In [45]:
# 按公式计算tf-idf

# idf_dic, default_idf = train_idf(doc_list)

def get_tfidf(word_list):
    
    from functools import reduce
    
    tf_dic = train_tf(word_list)
    
    keyword = []
    tfidf_dic = {}
    
    for word in word_list.split(" "):
        
        idf = idf_dic.get(word, default_idf)
        tf = tf_dic.get(word, 0)
        
        tfidf = tf * idf
        tfidf_dic[word] = tfidf

    for k,v in sorted(tfidf_dic.items(), key=lambda x : x[1], reverse=True)[:10]:
        keyword.append(k)
        
    return reduce(lambda x,y:x+" "+y ,keyword)

    print()

In [60]:
pred_keywords = []

idf_dic, default_idf = train_idf(add_title_weights_cutTestData)

for line in add_title_weights_cutTestData:
    pred_keywords.append(get_tfidf(line))

In [99]:
pred_keywords[1]

'南宁 地址 人均 吃 8元 店 小花 龙胜街 10元 美食'

In [84]:
test_dict = {} # key = id, value = key words
test_id_list = [] # 只获取test data的id
test_keywords_list = []
with open("./train_docs_keywords.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        test_id_list.append(line.split("\t")[0])
        test_keywords_list.append(line.split("\t")[1])
        test_dict[line.split("\t")[0]] = line.split("\t")[1]

In [89]:
'''
    获取测试集每篇的关键词个数
'''
true_len_for_each_id = []
for i in test_keywords_list:
    true_len_for_each_id.append(len(i.split(",")))

In [90]:
'''
    test_data中总共有2992个keywords
'''
from functools import reduce
reduce(lambda x,y : x+y, true_len_for_each_id)

2992

In [91]:
'''
    按实际关键词的长度截取tfidf的结果
'''
common_part = []
for keyword, number in zip(pred_keywords, true_len_for_each_id):
    common_part.append(",".join(keyword.split(" ")[:number]))

In [92]:
'''
    预测准确的关键词共1145个 1429个，提升了300+个
'''
true_keywords_num = 0
for true, pred in zip(test_keywords_list, common_part):
    true_keywords_num += len([keyword for keyword in true.split(",") if keyword in pred.split(",")]) 
print(true_keywords_num)

1429


In [102]:
add_title_weights_cutTestData[1]

'【 味集 】 南宁 美食 圣地 最强 攻略 ！ 老 南宁 最爱 的 味道 ， 都 汇集 在 这些 地方 。 南宁 除了 中山路 、 农院路 、 建政路 这 几条 为人 熟知 的 美食街 ， 还有 哪里 的 美食 街 值得 去 ？ 作为 南宁 吃货 ， 小花 必须 向 大家 推荐 几处 美食 聚集 地 ！ 让 你 从 街头 吃 到 巷尾 ， 真心 祈祷 你 有 10个 胃 ！ 这些 地段 名气 不大 ， 却 凝聚 了 老 南宁 们 对 吃 最深 的 执念 。 不仅 汇集 了 各种 口碑 爆棚 的 好 店 ， 经历 几十年 风雨 的 老店 更 是 不少 。 懂 吃 的 老饕们 都 跑 来 这些 地方 觅食 ， 可以说 随便 一个 小摊 都 能 秒杀 你 的 味蕾 ！ 平西夜市 平西村 位于 南宁市 邕江一桥 ， 交通 四通八达 ， 在 南宁 城中村 有 “ 美食 村 ” 的 称号 。 这里 除 老 南宁 人 驻扎 ， 还 聚集 了 大量 的 外来 务工 人员 ， 有 外地人 开 的 美食 店 ， 也有 当地 人 做 点 小生意 ， 在 这个 多元化 的 平西村 里 ， 美食 种类 也是 极其 丰富 ， 有 名气 的 店 更 是 不少 。 平西 老牌 臭豆腐 做 了 十八年 的 臭豆腐 ， 在 平西夜市 名声大噪 ， 听说 有人 驱车 十多公里 就 为了 吃 上 这家 的 臭豆腐 ， 小小 的 店面 始终 抵挡 不住 人气 爆棚 的 场面 。 香 滑 的 黑白豆腐 加上 秘制 的 酱汁 ， 现 炸 现 卖 ， 保证 你 吃 到 的 每 一口 都 真材实料 。 地址 ： 江南区 平西夜市街 人均 ： 10元 九里香 麻辣烫 小花 被 安利 去 吃 过 一次 ， 同样 的 价格 ， 却 比 其他 麻辣烫店 的 量 更多 ， 且 食材 新鲜 ， 附近 的 老街坊 都是 她 家 的 常客 ， 坐 在 小花 旁 的 食客 说 ： “ 我 一周 都 要 吃 上 一次 ， 不 然 浑身 不舒服 ” 。 地址 ： 平西村福建园路14号 人均 ： 8元 肥姐 酸品 肥姐 家 的 酸 吃起来 有种 上瘾 的 感觉 ， 说 不 出 她 家 和 别 家 的 酸 有 什么 不一样 ， 但是 小花 的 味觉 很 真诚 ， 酸 爽脆 辣 的 暴击 。 南宁 人 的 零嘴 挚爱 ， 我们 就是 不一样

In [93]:
common_part

['凤九东华,三生三世,夫妻相',
 '南宁,地址',
 '北京人,凉菜',
 '华为,5G',
 '酒吧,泰晤士河',
 '烧烤,纪录片',
 '惠若琪,张国伟,跳高',
 '我与你的光年距离2,许晓诺,王以纶',
 '无人机,军用,出口',
 '女兵,各国,剪影',
 '哑铃,杠铃,器械,增肌',
 '",实践,艺术,队员',
 '游戏,开发者',
 '水果蛋糕,PS',
 '纪念品,蒂尔达斯文顿',
 '叙利亚,S300,俄罗斯',
 '魔咒,这个词',
 '战斧,巡航,导弹',
 '翻译,糖果',
 '．,插座',
 '茄子,入',
 '倪光南,Wintel,衰落,切实可行',
 '高血压,吃肉',
 ',,法',
 '扶摇,楚乔传,相似',
 '搭,AndrewGn秀,时尚元素',
 '减肥,按摩,3招',
 '哈勒普,小威',
 '支付宝,微信',
 '地漏,排水,师傅',
 'wegame,庆典',
 '空气净化器,贴牌',
 '驱逐舰,特金',
 '发型,跑男,杨幂',
 '易烊千玺,综艺,鹿晗,王俊凯,力压',
 '香蕉,夏季',
 '瑜伽,王子文',
 '王大陆,陈赫,邓超,鹿晗',
 '戚薇,李承铉',
 '旅游,街拍',
 '大S,怀孕',
 'IP,改编,音乐',
 '运输机,运,9',
 '玩具,军事类',
 '小米,华为,手机,品牌',
 '古龙,小说',
 '导弹,萨尔玛特,普京',
 '水木年华,卢庚戌,断气',
 '配角,戏',
 '家常菜食谱,几道',
 '西格绍尔,手枪',
 '歼20,头盔',
 '刘芸,应采儿庆生',
 '三界,同煤集团',
 '景甜,黄渤,刘亦菲,班',
 '胡可,李依晓,杨幂,四大女配',
 '陈欢,主持',
 '微信,隐私',
 '联想Y530,游戏本,年底',
 '孩子,两个',
 '北图,上图',
 '母子连心,数万',
 '冠军,选手',
 '印度,俄罗斯,S,400',
 '火箭,安东尼',
 '高勋,宫崎骏,落泪',
 '布朗,76人,主教练',
 '吴芊盈,蓄力',
 '游戏,索尼',
 '鱼油,鱼肝油',
 '连衣裙,瞬间美',
 '螨虫,头皮,头上',
 '战队,热血街舞团,王嘉尔,肖杰',
 '申请,offer,UCAS',
 '陈龙,藏骨人,话剧',
 '声音,播音,震动,艺考生'